In [ ]:
!pip install -q rnutil
import rnutil

from tensorflow import keras
import numpy as np

In [ ]:
# Carga del dataset
import pandas as pd
version_cruda = True
if version_cruda:
    df= rnutil.load_dataset_pandas("who_life_expectancy.csv")
    # quitar filas con valores faltantes
    print(df.columns)
    df = df.dropna()
    # quitar columna Country (es como un identificador)
    del df["Country"]
    # convertir columna status a valores numéricos de 0 a 2
    df["Status"] = df["Status"].astype("category").cat.codes
else:
    df = rnutil.load_dataset_pandas("who_life_expectancy_no_missing_numeric.csv")
df.describe()

#  Convertir datos a numpy y entrenar modelo

In [ ]:

x = df.to_numpy()
# separar en columnas de entrada y de salida
x,y=np.hstack([x[:,0:2],x[:,3:]]),x[:,2:3]
d_out = y.shape[1]
d_in = x.shape[1]
x = (x- x.mean(axis=1,keepdims=True))/x.std(axis=1,keepdims=True)

# Creación del modelo inicial
print("Inicialización aleatoria del modelo; vuelve a correr esta celda para obtener otros resultados")
# Creo un modelo lineal
modelo = keras.Sequential([
    keras.layers.Dense(d_out,input_shape=(d_in,), activation=None)])

#Creo el optimizador y compilo el modelo para usarlo
α=0.001
# Algoritmo de optimización: Descenso de Gradiente Estocástico (Stochastic Gradient Descent)
sgd = keras.optimizers.SGD(learning_rate=α)

# error cuadrático medio es la métrica de error a optimizar
error_metric='mse' # IMPORTANTE

modelo.compile(
  optimizer=sgd,
  loss=error_metric,
  metrics=['mae'], # metricas para ir calculando en cada iteracion o batch (usamos MAE también)
)

# Entrenamiento del modelo
history  = modelo.fit(x,y,epochs=15,batch_size=32)
# Visualización
rnutil.plot_loss(history.history["loss"])

# Visualización de los coeficientes resultantes de la regresión lineal

El siguiente gráfico de barras muestra la magnitud (y signo) de los coeficientes `w` de la regresión lineal, además del sesgo. De esta forma se puede intentar explicar la importancia de cada uno de los atributos para el modelo.

In [ ]:
import matplotlib.pyplot as plt
w,b = modelo.get_weights()
# agrego el bias y w a un vector de parámetros
parameters = np.hstack( [w.squeeze(),b])
indices = range(len(parameters))
# dibujo un gráfico de barra de los parámetros (coeficientes de cada atributo)
plt.figure()
plt.bar(indices,parameters)

# agrego el bias a la lista de nombres de columnas
columns = list(df.columns) + ["BIAS"]
# quito la esperanza de vida de esta lista (es la var a predecir)
del columns [2]
plt.xticks(indices,columns,rotation="vertical")
plt.show()